### New install

In [1]:
!pip install --upgrade jupyterlab
!pip install --upgrade jupyter
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## Imports

In [2]:
import logging
import sys
sys.path.append('../../src')
import timm
import torch
import torch.nn as nn
import torchaudio.transforms as AT
import torchvision.transforms as VT
from nnAudio import features

import config

Created directory: /dli/data/plots
Created directory: /dli/data/features/MED
Created directory: /dli/models


/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:


def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [4]:
def load_model(model_file,model = timm.create_model('convnext_small',pretrained=True, num_classes=2, in_chans=1, drop_path_rate=0.05, global_pool='max', drop_rate=0.05)):
    model = model
    state_dict = torch.load(model_file)
    model.load_state_dict(state_dict)
    model.train()
    return model

In [13]:

#model = timm.create_model('convnext_small',pretrained=True, num_classes=2, in_chans=1, drop_path_rate=0.05, global_pool='max', drop_rate=0.05)


## Load the base fp-32 model

In [10]:
!pwd

/dli/task


In [6]:
class MyModel(nn.Module):
    def __init__(self, model_name, image_size = 224):
        super().__init__()
        # num_classes=0 removes the pretrained head
        self.backbone = timm.create_model(model_name,
                        pretrained=True, num_classes=2, in_chans=1, 
                        drop_path_rate=0.2, global_pool='max',
                        drop_rate=0.25)
        #####  This section is model specific
        #### It freezes some fo the layers by name
        #### you'll have to inspect the model to see the names
                #### end layer freezing
        self.out = nn.Linear(self.backbone.num_features, 1)
        self.sizer = VT.Resize((image_size,image_size),antialias = True)
        self.spec_layer = AT.Spectrogram(n_fft = int(config.NFFT), return_complex= False,).to('cuda')
        self.batch_norm = nn.BatchNorm2d(num_features= 1).to('cuda')
        #self.augment_layer = augment_audio(trainable = True, sample_rate = config.rate)
        self.quant = torch.ao.quantization.QuantStub()
        self.dequant = torch.ao.quantization.DeQuantStub()
        
    def forward(self, x):
        # first compute spectrogram
        x = self.quant(x)
        spec_gram = self.spec_layer(x)
        #print("post spec gram shape = ",spec_gram.shape)
        spec_gram = self.batch_norm(spec_gram)
        #print("post norm shape = ",spec_gram.shape)
        spec_gram_nan_check = torch.isnan(spec_gram).any().item()
        assert not (spec_gram_nan_check) ,"Tensor contains NaN values after spec gram creation."
             
        
        x = self.sizer(spec_gram.squeeze(dim = 1))
        #print("post sizer shape = ",x.shape)
        x = x.unsqueeze(dim = 1)
        #print("post unsqueeze shape = ",x.shape)
        
        # then repeat channels
        del spec_gram,spec_gram_nan_check
                
        x = self.backbone(x)
        #backbone_op_nan_check = torch.isnan(x).any().item()
        #assert not (backbone_op_nan_check) ,"Tensor contains NaN values in the backbone OP "
        #print("x shape = " + str(x.shape))
        #print("x = " +str(x))
        #pred = nn.Softmax(x)
        x = self.dequant(x)
        output = x
        #print(np.argmax(pred.detach().cpu().numpy()))
        #print(pred)
        #print(output)
        del x 
        return output


In [7]:
import os
filename = os.path.join("models","model_med0_2023_11_18_04_37_20.pth")

#state_dict = torch.load(filename)
model_fp32  = load_model(filename, model = MyModel('convnext_small',224))





/opt/conda/lib/python3.8/site-packages/torchaudio/transforms/_transforms.py:94: UserWarning: `return_complex` argument is now deprecated and is not effective.`torchaudio.transforms.Spectrogram(power=None)` always returns a tensor with complex dtype. Please remove the argument in the function call.
  warnings.warn(


In [6]:
#let's try to save the float model
#model =MyModel('convnext_small',224)

/opt/conda/lib/python3.8/site-packages/torchaudio/transforms/_transforms.py:94: UserWarning: `return_complex` argument is now deprecated and is not effective.`torchaudio.transforms.Spectrogram(power=None)` always returns a tensor with complex dtype. Please remove the argument in the function call.
  warnings.warn(


In [75]:
def save_torchscript_model(model, model_dir, model_filename):

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.jit.save(torch.jit.script(model), model_filepath)

## Quantization starts here

In [8]:
model_fp32.eval()
model_fp32.to('cpu')
model_fp32.qconfig = torch.ao.quantization.get_default_qconfig('x86')
from torch.quantization import convert


In [10]:
model_fp32_prepared = torch.ao.quantization.prepare(model_fp32)

/opt/conda/lib/python3.8/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [11]:
model_int8_conv_n = torch.ao.quantization.convert(model_fp32_prepared)

/opt/conda/lib/python3.8/site-packages/torch/ao/quantization/observer.py:1207: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [12]:
torch.save(model_int8_conv_n, "model_int8_conv.pt")

In [10]:
#torch.jit.script(model_int8 ,"convnext_ft_quant_int8.pt")

/opt/conda/lib/python3.8/site-packages/torch/jit/_script.py:1277: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn(


RecursiveScriptModule(
  original_name=ConvNeXt
  (stem): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Conv2d)
    (1): RecursiveScriptModule(original_name=LayerNorm2d)
  )
  (stages): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=ConvNeXtStage
      (downsample): RecursiveScriptModule(original_name=Identity)
      (blocks): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(
          original_name=ConvNeXtBlock
          (conv_dw): RecursiveScriptModule(original_name=Conv2d)
          (norm): RecursiveScriptModule(original_name=LayerNorm)
          (mlp): RecursiveScriptModule(
            original_name=Mlp
            (fc1): RecursiveScriptModule(original_name=Linear)
            (act): RecursiveScriptModule(original_name=GELU)
            (drop1): RecursiveScriptModule(original_name=Dropout)
            (norm): RecursiveScriptModule(o

In [9]:
#model_int8_new_t_no_qnt_layer = torch.quantization.convert(model_fp32_prepared.to('cpu').eval())

/opt/conda/lib/python3.8/site-packages/torch/ao/quantization/utils.py:317: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


In [13]:
torch.jit.save(torch.jit.script(model_int8_conv_n), "conv_quant_int8.pt")